# Run through PACS and create label map from ImageNet labels to PACs labels

Finding: Turns out that Giraffe is not available in ImageNet1k, so cannot use a mapper easily (at least hard to argue why that is a valid idea).

In [1]:
import blackhc.project.script

Appended /home/blackhc/PycharmProjects/gde_repro/src to paths
Switched to directory /home/blackhc/PycharmProjects/gde_repro
%load_ext autoreload
%autoreload 2


In [2]:
import timm

In [3]:
model = timm.create_model("beit_large_patch16_224", pretrained=True)
model.eval()

/home/blackhc/anaconda3/envs/gde_repro/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484808560/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Downloading: "https://conversationhub.blob.core.windows.net/beit-share-public/beit/beit_large_patch16_224_pt22k_ft22kto1k.pth" to /home/blackhc/.cache/torch/hub/checkpoints/beit_large_patch16_224_pt22k_ft22kto1k.pth


Beit(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate=none)
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): Layer

In [4]:
import hub

In [6]:
from torchvision import transforms
from timm.data.transforms_factory import transforms_imagenet_eval

In [7]:
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

In [11]:
model.cuda()

Beit(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate=none)
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): Layer

In [9]:
model.pretrained_cfg

{'url': 'https://conversationhub.blob.core.windows.net/beit-share-public/beit/beit_large_patch16_224_pt22k_ft22kto1k.pth',
 'num_classes': 1000,
 'input_size': (3, 224, 224),
 'pool_size': None,
 'crop_pct': 0.9,
 'interpolation': 'bicubic',
 'fixed_input_size': True,
 'mean': (0.5, 0.5, 0.5),
 'std': (0.5, 0.5, 0.5),
 'first_conv': 'patch_embed.proj',
 'classifier': 'head',
 'architecture': 'beit_large_patch16_224'}

In [8]:
cfg=model.pretrained_cfg
eval_transform = transforms_imagenet_eval(img_size=cfg["input_size"][1], crop_pct=1, interpolation=cfg["interpolation"], mean=cfg["mean"], std=cfg["std"], use_prefetcher=False)
print(eval_transform)
#data_config = resolve_data_config(dict(crop_pct=0.), model=model)
#eval_transform=create_transform(**data_config)
eval_transform.transforms = [transforms.ToPILImage()] + eval_transform.transforms

Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=tensor([0.5000, 0.5000, 0.5000]), std=tensor([0.5000, 0.5000, 0.5000]))
)


In [10]:
import hub
ds = hub.load('hub://activeloop/pacs-train')

hub://activeloop/pacs-train loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/pacs-train


In [12]:
import torch
import torch.nn

In [23]:
eval_dataloader = ds.pytorch(batch_size=128, transform=dict(images=eval_transform, labels=None), tensors=["images", "labels"], shuffle=False, pin_memory=True, num_workers=8, use_progress_bar=True, use_local_cache=True)

hub://activeloop/pacs-train:   0%|                                                                                                                                                                                                                                                               | 0/70 [00:00<?, ?it/s]

In [20]:
from typing import Any

import torch


class SafeModule(torch.nn.Module):
    dtype: Any
    tdevice: torch.device

    def __init__(self, dtype=None, device=None):
        super().__init__()

        self.dtype = dtype or torch.float32
        self.tdevice = device or torch.device("cpu")

    def convert_tensor(self, tensor: torch.Tensor, *, device_only=False, non_blocking=False):
        return tensor.to(device=self.tdevice, dtype=self.dtype if not device_only else None, non_blocking=non_blocking)

    def safe_forward(self, *inputs, **kwargs):
        return inputs

    def set_device(self, device):
        for module in self.modules():
            if isinstance(module, SafeModule):
                module.tdevice = device

        if device == torch.device("cpu"):
            return super().cpu()
        return super().cuda(device)

    def cuda(self, device=None):
        return self.set_device(device if device else torch.device("cuda"))

    def cpu(self):
        return self.set_device(torch.device("cpu"))

    def forward(self, *inputs, **kwargs):
        inputs = [
            input.to(device=self.tdevice, dtype=self.dtype) if isinstance(input, torch.Tensor) else input
            for input in inputs
        ]
        kwargs = {
            name: arg.to(device=self.tdevice, dtype=self.dtype) if isinstance(arg, torch.Tensor) else arg
            for name, arg in kwargs.items()
        }

        return self.safe_forward(*inputs, **kwargs)

    def type(self, dst_type):
        for module in self.modules():
            if isinstance(module, SafeModule):
                module.dtype = dst_type

        # Now update the actual parameters and buffers.
        super().type(dst_type)
        return self

    def float(self):
        return self.type(torch.float32)

    def double(self):
        return self.type(torch.float64)

    def half(self):
        return self.type(torch.float16)

    def to(self, *args, **kwargs):
        device, dtype, non_blocking, convert_to_format = torch._C._nn._parse_to(*args, **kwargs)

        if device is not None:
            self.tdevice = device

        if dtype is not None:
            self.dtype = dtype

        return super().to(*args, **kwargs)

In [21]:
class CategoricalMap(SafeModule):
    """Keeps track of relevant quantities, so we can compute any kind of information quantity after the fact."""

    acc_p_Y_Z: torch.Tensor
    num_x: int

    in_capacity: int
    out_capacity: int

    def __init__(self, in_capacity: int, out_capacity: int, dtype=torch.float64, device=None):
        super().__init__(dtype=dtype, device=device)

        self.out_capacity = out_capacity
        self.in_capacity = in_capacity
        self.num_x = 0

        self.register_buffer(
            "acc_p_Y_Z", torch.zeros((out_capacity, in_capacity), dtype=dtype, device=device, requires_grad=False)
        )
        self.register_buffer("acc_h_Z__X", torch.zeros((), dtype=dtype, device=device, requires_grad=False))

    def safe_forward(self, encoding):
        # predictions: BxC
        normalized_predictions = torch.nn.functional.softmax(encoding, dim=-1, dtype=self.dtype)
        p_Y__Z = self.get_p_Y__Z()
        return torch.log(p_Y__Z @ normalized_predictions.t()).t()

    def reset(self):
        with torch.no_grad():
            self.acc_p_Y_Z.zero_()
            self.num_x = 0

    def fit(self, logits_x_z: torch.Tensor, labels_x: torch.Tensor):
        with torch.no_grad():
            logits_x_z = logits_x_z.to(dtype=self.dtype, device=self.tdevice)
            p_X_Z = torch.nn.functional.softmax(logits_x_z, dim=1, dtype=self.dtype)

            labels_x = labels_x.to(dtype=self.dtype, device=self.tdevice)

            # TODO: replace with one_hot(labels_x, num_classes=self.out_capacity)?
            i_y_x = torch.tensor(range(self.out_capacity), device=labels_x.device, requires_grad=False).reshape(
                (-1, 1)
            ) == labels_x.reshape((1, -1))
            p_Y_Z = (i_y_x.double() @ p_X_Z).detach()
            self.acc_p_Y_Z += p_Y_Z

            self.num_x += len(labels_x)
            
    def get_p_Y_Z(self):
        return self.acc_p_Y_Z / self.num_x
    
    def get_p_Y__Z(self):
        return self.acc_p_Y_Z / self.acc_p_Y_Z.sum(dim=0, keepdim=True)

In [26]:
pacs_mapper = CategoricalMap(1000,7, device="cuda")

In [27]:
from tqdm.auto import tqdm

In [28]:
with torch.inference_mode():
    for batch_images, batch_labels in tqdm(eval_dataloader):
        logits_x_z = model(batch_images.cuda())
        pacs_mapper.fit(logits_x_z, batch_labels)

  0%|          | 0/71 [00:00<?, ?it/s]

In [30]:
p_Y_Z = pacs_mapper.get_p_Y_Z()

In [31]:
p_Y_Z.shape

torch.Size([7, 1000])

In [46]:
p_Y_Z[2].topk(10)

torch.return_types.topk(
values=tensor([0.0101, 0.0101, 0.0030, 0.0027, 0.0026, 0.0024, 0.0023, 0.0016, 0.0016,
        0.0016], device='cuda:0', dtype=torch.float64),
indices=tensor([353, 355, 917, 921, 340, 354, 549, 684, 446, 352], device='cuda:0'))

In [45]:
(p_Y_Z/p_Y_Z.sum(dim=1, keepdim=True))[:, 235]

tensor([0.0351, 0.0004, 0.0013, 0.0003, 0.0018, 0.0003, 0.0003],
       device='cuda:0', dtype=torch.float64)

In [42]:
p_Y_Z.sum(dim=1)

tensor([0.1732, 0.1655, 0.1567, 0.1114, 0.1542, 0.0942, 0.1447],
       device='cuda:0', dtype=torch.float64)